In [52]:
# Import Dependencies
from selenium import webdriver
import pandas as pd 
from sqlalchemy import create_engine
from StatPass import username, password

In [2]:
# Stathead/Basket-ball Reference requires a login
# We will send the Browser to a login page that will automatically redirect to our starting point
login_address = 'https://stathead.com/users/login.cgi?token=1&redirect_uri=https%3A//www.basketball-reference.com/leagues/NBA_2020_totals.html'

In [3]:
# Create Browser and set it to open the page in Chrome
browser = webdriver.Chrome(executable_path=r'chromedriver.exe')
browser.get(login_address)

# Fill in the form with username and password
user_name = browser.find_element_by_id('username')
user_name.send_keys(username)
pass_word = browser.find_element_by_id('password')
pass_word.send_keys(password)

# Click the login button
browser.find_element_by_id('login').submit()

In [78]:
# Confirm page address after log-in redirect
browser.current_url

'https://www.basketball-reference.com/leagues/NBA_2020_totals.html'

In [79]:
# Define the year of data we want to start looping
start_year = 2020 # the season 2020 is for the 2019-20 season, the last year of complete data

# define the amount of years we want to loop
for x in range(1, 6):
    site_address = browser.current_url
    year_df = pd.read_html(site_address)[0] # our data is the first table on the page
    year_df['Season'] = start_year
    year_df.to_csv(f'NBA_By_Year/{start_year}Players.csv') # after extracting, save to csv by year
    del year_df # delete the DataFrame from memory
    start_year -= 1 # change the year to get next
    browser.get(f'https://www.basketball-reference.com/leagues/NBA_{start_year}_totals.html') # navigate to the next year's data

    
    

In [80]:
# Confirm year_df is not stored in memory
year_df

NameError: name 'year_df' is not defined

### Merge all CSVs into a DataFrame

In [94]:
# Start combined_df with one year
one_df = pd.read_csv('NBA_By_Year/2020Players.csv', index_col=0)

In [95]:
# View the DataFrame
one_df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,207,376,583,146,51,67,94,122,684,2020
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,176,559,735,368,82,93,204,182,1146,2020
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,103,289,392,129,36,87,74,128,1001,2020
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,2,1,3,0,0,0,1,1,2,2020
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,525,Trae Young,PG,21,ATL,60,60,2120,546,1249,...,32,223,255,560,65,8,289,104,1778,2020
673,526,Cody Zeller,C,27,CHO,58,39,1341,251,479,...,160,251,411,88,40,25,75,140,642,2020
674,527,Tyler Zeller,C,30,SAS,2,0,4,1,4,...,3,1,4,0,0,0,0,0,2,2020
675,528,Ante Žižić,C,23,CLE,22,0,221,41,72,...,18,48,66,6,7,5,10,27,96,2020


### Initial Observations:
- There are names with special characters (eg. #675 Ante Žižić)
- The index doesn't match the Rk (rank). Upon investigation there are two causes:
    - Every 20 rows there is a new header row
    - Players who played on multiple teams will have multiple lines (1 per team and 1 totals) all with the same Rk#
    - Will need to check for null values

In [16]:
# If we have a list of file names, can we loop through and merge?
files_list = ['NBA_By_Year/2020Players.csv', 'NBA_By_Year/2019Players.csv', 'NBA_By_Year/2018Players.csv', 'NBA_By_Year/2017Players.csv', 'NBA_By_Year/2016Players.csv']

# code idea from https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
df_from_each_file = (pd.read_csv(f) for f in files_list)
all_df = pd.concat(df_from_each_file, ignore_index=True)

In [17]:
all_df

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,0,1,Steven Adams,C,26,OKC,63,63,1680,283,...,207,376,583,146,51,67,94,122,684,2020
1,1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,...,176,559,735,368,82,93,204,182,1146,2020
2,2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,...,103,289,392,129,36,87,74,128,1001,2020
3,3,4,Kyle Alexander,C,23,MIA,2,0,13,1,...,2,1,3,0,0,0,1,1,2,2020
4,4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,596,472,Joe Young,PG,23,IND,41,0,384,62,...,6,44,50,65,15,0,33,30,154,2016
3317,597,473,Nick Young,SG,30,LAL,54,2,1033,126,...,14,83,97,34,23,7,30,50,392,2016
3318,598,474,Thaddeus Young,PF,27,BRK,73,73,2407,495,...,176,484,660,135,112,37,136,182,1102,2016
3319,599,475,Cody Zeller,C,23,CHO,73,60,1774,231,...,138,317,455,71,57,63,68,204,638,2016


### Additional Observations:
- We don't need the "Unnamed: 0" column
- The Rk should be kept until the last moment, as it will be useful in cleaning the data in combination with the Season column
- Instead of hand typing the files_list, create it as part of the for loop

In [20]:
# Let's get the next 25 years of data
for x in range(1, 26):
    site_address = browser.current_url
    year_df = pd.read_html(site_address)[0]
    year_df['Season'] = start_year
    year_df.to_csv(f'NBA_By_Year/{start_year}Players.csv')
    files_list.append(f'NBA_By_Year/{start_year}Players.csv') # append file name to files_list
    del year_df # delete the DataFrame from memory
    start_year -= 1 # change the year to get next
    browser.get(f'https://www.basketball-reference.com/leagues/NBA_{start_year}_totals.html') # navigate to the next year's data

In [4]:
# Close the browser session
browser.quit()

# Delete the existing df and build from scratch with all files
del all_df
del df_from_each_file

NameError: name 'all_df' is not defined

Only use the following cell if not running file from the top

In [5]:
# Create files_list loop so don't need to re-run the browser loop every time we open the file
files_list = []
start_year = 2020
for x in range(1, 31):
    files_list.append(f'NBA_By_Year/{start_year}Players.csv') # append file name to files_list
    start_year -= 1 # change the year to get next

In [6]:
df_from_each_file = (pd.read_csv(f) for f in files_list)
all_df = pd.concat(df_from_each_file, ignore_index=True)

## Transform the Data

In [7]:
# Let's see all_df
all_df

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,0,1,Steven Adams,C,26,OKC,63,63,1680,283,...,207,376,583,146,51,67,94,122,684,2020
1,1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,...,176,559,735,368,82,93,204,182,1146,2020
2,2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,...,103,289,392,129,36,87,74,128,1001,2020
3,3,4,Kyle Alexander,C,23,MIA,2,0,13,1,...,2,1,3,0,0,0,1,1,2,2020
4,4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17339,455,385,Howard Wright,PF,23,ATL,4,0,20,2,...,1,5,6,0,0,0,2,3,5,1991
17340,456,385,Howard Wright,PF,23,ORL,8,0,136,15,...,10,27,37,3,3,5,9,23,43,1991
17341,457,385,Howard Wright,PF,23,DAL,3,0,8,2,...,1,1,2,0,1,0,0,2,6,1991
17342,458,386,A.J. Wynder,PG,26,BOS,6,0,39,3,...,1,2,3,8,1,0,4,1,12,1991


In [8]:
# Drop the Unnamed column
all_df = all_df.drop(columns={'Unnamed: 0'})

In [9]:
# Get rid of additional header rows
all_df = all_df.loc[all_df["Rk"] != "Rk"]

In [10]:
# Get rid of multiple player lines
# Let's analyze first - Trevor Ariza, the most traded player of all time!
all_df.loc[(all_df["Player"] == "Trevor Ariza")]

# Note that the Totals of a player is always first

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
18,19,Trevor Ariza,SF,34,TOT,53,21,1493,142,324,...,34,212,246,91,69,15,56,111,423,2020
19,19,Trevor Ariza,SF,34,SAC,32,0,791,64,165,...,21,125,146,50,35,7,28,63,192,2020
20,19,Trevor Ariza,SF,34,POR,21,21,702,78,159,...,13,87,100,41,34,8,28,48,231,2020
702,23,Trevor Ariza,SF,33,TOT,69,69,2349,294,736,...,50,321,371,252,91,21,106,130,863,2019
703,23,Trevor Ariza,SF,33,PHO,26,26,884,86,227,...,16,129,145,87,38,8,39,44,258,2019
704,23,Trevor Ariza,SF,33,WAS,43,43,1465,208,509,...,34,192,226,165,53,13,67,86,605,2019
1430,20,Trevor Ariza,SF,32,HOU,67,67,2269,268,651,...,33,261,294,105,98,13,52,132,782,2018
2123,19,Trevor Ariza,SF,31,HOU,80,80,2773,326,798,...,54,404,458,175,147,20,73,133,936,2017
2746,24,Trevor Ariza,SF,30,HOU,81,81,2859,357,859,...,67,299,366,188,160,26,113,177,1025,2016
3347,22,Trevor Ariza,SF,29,HOU,82,82,2930,366,910,...,77,382,459,209,152,17,141,186,1048,2015


In [11]:
all_df= all_df.drop_duplicates(subset=['Rk', 'Season'], keep='first')

In [12]:
all_df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,207,376,583,146,51,67,94,122,684,2020
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,176,559,735,368,82,93,204,182,1146,2020
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,103,289,392,129,36,87,74,128,1001,2020
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,2,1,3,0,0,0,1,1,2,2020
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,383,Haywoode Workman,PG,25,WSB,73,56,2034,234,515,...,51,191,242,353,87,7,135,162,581,1991
17337,384,James Worthy*,SF,29,LAL,78,74,3008,716,1455,...,107,249,356,275,104,35,127,117,1670,1991
17338,385,Howard Wright,PF,23,TOT,15,0,164,19,47,...,12,33,45,3,4,5,11,28,54,1991
17342,386,A.J. Wynder,PG,26,BOS,6,0,39,3,12,...,1,2,3,8,1,0,4,1,12,1991


In [13]:
# Data Integrity: Let's analyze our data types
all_df.dtypes

Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
Season     int64
dtype: object

In [14]:
# Get column names
all_df.keys()

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Season'],
      dtype='object')

In [15]:
# Convert columns to correct type 

text_keys = ['Player','Pos','Tm']
integers = ['Age','G', 'GS', 'MP', 'FG', 'FGA','3P', '3PA',  '2P', '2PA', 'FT', 'FTA', 
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Season']
percents = ['FG%','3P%','2P%','FT%', 'eFG%']

# Loop through to convert
for key in percents:
    all_df[key] = pd.to_numeric(all_df[key])
for key in integers:
    all_df[key] = pd.to_numeric(all_df[key])

In [16]:
# Check to make sure conversion worked
all_df.dtypes

Rk         object
Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP          int64
FG          int64
FGA         int64
FG%       float64
3P          int64
3PA         int64
3P%       float64
2P          int64
2PA         int64
2P%       float64
eFG%      float64
FT          int64
FTA         int64
FT%       float64
ORB         int64
DRB         int64
TRB         int64
AST         int64
STL         int64
BLK         int64
TOV         int64
PF          int64
PTS         int64
Season      int64
dtype: object

In [17]:
# Data Integrity: Let's check for completeness by Games per Season - should be 82
game_starters= all_df.groupby('Season')[('GS')].sum() # 5 starters per team per game
games_per_season = game_starters / 10

# count teams per season (currently 30, but there has been expansion. Subtract 1 for "TOT")
teams = all_df.groupby('Season')['Tm'].nunique() - 1
# Calculate games per season (divide by 2 as there are two teams per game)
games_per_season/(teams/2)


Season
1991    82.000000
1992    82.000000
1993    82.000000
1994    82.000000
1995    82.000000
1996    82.000000
1997    82.000000
1998    82.000000
1999    50.000000
2000    82.000000
2001    82.000000
2002    82.000000
2003    82.000000
2004    82.000000
2005    82.000000
2006    82.000000
2007    82.000000
2008    82.000000
2009    82.000000
2010    82.000000
2011    82.000000
2012    66.000000
2013    81.933333
2014    82.000000
2015    82.000000
2016    82.000000
2017    82.000000
2018    82.000000
2019    82.000000
2020    70.600000
dtype: float64

#### Outliers above:
- 1999: Lockout shortened season
- 2012: Lockout shortened season
- 2013: Cancellation of the scheduled 4/16 game Celtics/Pacers after the Boston Marathon bombing
- 2020: Covid-19 shortened season - non-round amount as teams played different amount of games

### Nulls
As you can see below, it appears all nulls are in "%" columns.

We will check to make sure this is true before casting all NaN to 0.

In [18]:
all_df[pd.isnull(all_df).any(axis=1)]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,2,1,3,0,0,0,1,1,2,2020
11,12,Ryan Anderson,C,31,HOU,2,0,14,2,7,...,0,7,7,2,1,0,1,1,5,2020
13,14,Kostas Antetokounmpo,PF,22,LAL,5,0,20,3,3,...,2,1,3,2,0,0,1,2,7,2020
58,45,Bismack Biyombo,C,27,CHO,53,29,1029,151,278,...,120,188,308,50,10,49,48,116,393,2020
67,52,Marques Bolden,C,21,CLE,1,0,3,0,0,...,0,2,2,0,1,0,0,1,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17278,332,Jim Thomas,SG,30,MIN,3,0,14,1,4,...,0,0,0,1,1,0,1,0,2,1991
17282,336,Bob Thornton,PF,28,MIN,12,1,110,4,13,...,1,14,15,1,0,3,9,18,16,1991
17298,351,Stojko Vranković,C,27,BOS,31,0,166,24,52,...,15,36,51,4,1,29,24,43,58,1991
17305,358,Mark West,C,30,PHO,82,64,1957,247,382,...,171,393,564,37,32,161,86,266,629,1991


In [19]:
# check Kyle Alexander 3Pt% (row 3, column 13)
all_df.iloc[3,13]

nan

In [20]:
# Create DF without percent rows
no_percents = all_df.drop(columns={'FG%','3P%','2P%','FT%', 'eFG%'})
no_percents

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,207,376,583,146,51,67,94,122,684,2020
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,176,559,735,368,82,93,204,182,1146,2020
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,103,289,392,129,36,87,74,128,1001,2020
3,4,Kyle Alexander,C,23,MIA,2,0,13,1,2,...,2,1,3,0,0,0,1,1,2,2020
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,383,Haywoode Workman,PG,25,WSB,73,56,2034,234,515,...,51,191,242,353,87,7,135,162,581,1991
17337,384,James Worthy*,SF,29,LAL,78,74,3008,716,1455,...,107,249,356,275,104,35,127,117,1670,1991
17338,385,Howard Wright,PF,23,TOT,15,0,164,19,47,...,12,33,45,3,4,5,11,28,54,1991
17342,386,A.J. Wynder,PG,26,BOS,6,0,39,3,12,...,1,2,3,8,1,0,4,1,12,1991


In [21]:
# Check null values in noo-percents DF
no_percents[pd.isnull(no_percents).any(axis=1)]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season


In [22]:
del no_percents

In [23]:
# Make null values 0
all_df = all_df.fillna(0)

In [24]:
# check Kyle Alexander 3Pt% (row 3, column 13) - Reprise
all_df.iloc[3,13]

0.0

### Special Characters Analysis

In [25]:
# Create list of special characters to look for in name series
spec_chars = ["!","#","%","&","+",",","/",":",";","<",
              "=",">","?","@","^","_",
              "`","~","–", "ć"]



In [26]:
# Create Series of player names
player_names = all_df["Player"]

In [27]:
# Explore special characters 
players = []
for x in player_names:
    for y in spec_chars:
        if y in x:
            players.append(x)
            pass
    continue

# Visualize to see potential issues
players

['Bogdan Bogdanović',
 'Bojan Bogdanović',
 'Luka Dončić',
 'Goran Dragić',
 'Nikola Jokić',
 'Boban Marjanović',
 'Jusuf Nurkić',
 'Luka Šamanić',
 'Dario Šarić',
 'Alen Smailagić',
 'Nikola Vučević',
 'Ante Žižić',
 'Bogdan Bogdanović',
 'Bojan Bogdanović',
 'Luka Dončić',
 'Goran Dragić',
 'Nikola Jokić',
 'Boban Marjanović',
 'Nikola Mirotić',
 'Jusuf Nurkić',
 'Dario Šarić',
 'Miloš Teodosić',
 'Nikola Vučević',
 'Ante Žižić',
 'Bogdan Bogdanović',
 'Bojan Bogdanović',
 'Goran Dragić',
 'Nikola Jokić',
 'Boban Marjanović',
 'Nikola Mirotić',
 'Jusuf Nurkić',
 'Dario Šarić',
 'Mirza Teletović',
 'Miloš Teodosić',
 'Nikola Vučević',
 'Ante Žižić',
 'Bojan Bogdanović',
 'Goran Dragić',
 'Nikola Jokić',
 'Boban Marjanović',
 'Nikola Mirotić',
 'Jusuf Nurkić',
 'Dario Šarić',
 'Mirza Teletović',
 'Nikola Vučević',
 'Sasha Vujačić',
 'Bojan Bogdanović',
 'Goran Dragić',
 'Nikola Jokić',
 'Boban Marjanović',
 'Nikola Mirotić',
 'Jusuf Nurkić',
 'Nikola Peković',
 'Mirza Teletović',
 'Nik

### Decision: remove * from names, keep everything else

- "*" represents players in the hall of fame. Need to remove this character
- "-", "." and "'" common name features. removed from spec_char list
- Name characters like ž,ć, etc. seem to be on wikipedia as well - we will not remove

In [28]:
# Create new list of players without asterisks
clean_players = []
for player in player_names:
    clean_players.append(player.replace('*', ''))

In [29]:
# Look for James Worthy - per the views above he had an asterisk!
clean_players[-5:]

['Haywoode Workman',
 'James Worthy',
 'Howard Wright',
 'A.J. Wynder',
 'Danny Young']

In [30]:
# Replace the Player column with the cleaned up names
all_df['Player'] = clean_players

In [31]:
# No more data cleaning to do - let's drop the Rk column and reset the index
all_df = all_df.drop(columns={"Rk"})

In [32]:
# Reset index
all_df = all_df.reset_index()

In [33]:
# Drop index column
all_df = all_df.drop(columns={'index'})

In [34]:
# Reflect all_df to be merged with countries
all_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,Steven Adams,C,26,OKC,63,63,1680,283,478,0.592,...,207,376,583,146,51,67,94,122,684,2020
1,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,0.557,...,176,559,735,368,82,93,204,182,1146,2020
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,0.493,...,103,289,392,129,36,87,74,128,1001,2020
3,Kyle Alexander,C,23,MIA,2,0,13,1,2,0.500,...,2,1,3,0,0,0,1,1,2,2020
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,0.368,...,9,75,84,89,17,8,54,57,267,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,Haywoode Workman,PG,25,WSB,73,56,2034,234,515,0.454,...,51,191,242,353,87,7,135,162,581,1991
13560,James Worthy,SF,29,LAL,78,74,3008,716,1455,0.492,...,107,249,356,275,104,35,127,117,1670,1991
13561,Howard Wright,PF,23,TOT,15,0,164,19,47,0.404,...,12,33,45,3,4,5,11,28,54,1991
13562,A.J. Wynder,PG,26,BOS,6,0,39,3,12,0.250,...,1,2,3,8,1,0,4,1,12,1991


**Web Scraping**
* We want to get the home country of every player in the NBA
* Wikipedia has a list of all the international players in the NBA 
* We can assume that all other players are from the USA

In [35]:
# Set the website url
url = 'https://en.wikipedia.org/wiki/List_of_foreign_NBA_players'

In [36]:
# use read_html to scrape out tables and select the relevant table
tables = pd.read_html(url)[6]
tables

,Nationality[A],Birthplace[B],Player,Pos.,Career[C],Yrs,Notes,Ref.
0,Angola,—,Bruno Fernando*,G/F,2019–present,2,—,[11]
1,Antigua and Barbuda,United States,Julius Hodge,G,2005–2007,2,"Born in the United States, became a naturalize...",[12]
2,Argentina,—,Nicolás Brussino,G/F,2016–2017,2,—,[13]
3,Argentina,—,Facundo Campazzo*,G,2020–present,1,—,[14]
4,Argentina,—,Carlos Delfino,G,2004–2008;2009–2014,8,—,[15]
...,...,...,...,...,...,...,...,...
583,Venezuela,United States,Askia Jones,G,1994,1,"Born in the United States, represented Venezue...",[754]
584,Venezuela,United States,Harold Keeling,F,1986,1,"Born in the United States, represented Venezue...",[755]
585,Venezuela,United States,Donta Smith,G,2004–2006,2,"Born on the United States, represented Venezuela.",[756]
586,Venezuela,—,Óscar Torres,F,2001–2002,2,—,[757]


In [37]:
# drop irrelevant columns
tables = tables.drop(columns = ['Pos.', 'Yrs', 'Notes', 'Ref.'])

**Insights**

In order to select international players, we look at the birthplace of the players. However, in a lot of cases the column is blank, in which case, we look at their nationality (as several players are naturalized).

To do this, we run a for loop to select birthplace where given, else nationality where it is blank.

For this, we have converted the two columns into lists, appended into a new list, and then added back a new country column that has all the players' country

In [38]:
# create lists out of nationality and birthplace columns in order to iterrate through it
nationality = tables['Nationality[A]'].to_list()
birthplace = tables['Birthplace[B]'].to_list()

In [39]:
# define an empty country list
# run a loop to iterrate through nationality and birthplace and append to country list
country = []
for x in range(len(nationality)):
    if birthplace[x] == '—':
        country.append(nationality[x])
    else: 
        country.append(birthplace[x])

In [40]:
country

['Angola',
 'United States',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Argentina',
 'Sudan(now South Sudan)',
 'Australia',
 'Australia',
 'Australia',
 'New Zealand',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'United States',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Australia',
 'Sudan(now South Sudan)',
 'Sudan(now South Sudan)',
 'United States',
 'Australia',
 'Australia',
 'Australia',
 'Austria',
 'Bahamas',
 'Bahamas',
 'Bahamas',
 'Bahamas',
 'Bahamas',
 'United States',
 'Zaire(now Democratic Republic of the Congo)',
 'United States',
 'United States',
 'United States',
 'United States',
 'Bosnia and Herzegovina',
 'Republic of Bosnia and Herzegovina',
 'SFR Yugoslavia(now Montenegro)',
 'SFR Yugoslavia(now Bosnia and Herzegovina)',
 'SFR Yugoslavia(n

In [41]:
# add a new country column from the list
tables['Country'] = country
tables

,Nationality[A],Birthplace[B],Player,Career[C],Country
0,Angola,—,Bruno Fernando*,2019–present,Angola
1,Antigua and Barbuda,United States,Julius Hodge,2005–2007,United States
2,Argentina,—,Nicolás Brussino,2016–2017,Argentina
3,Argentina,—,Facundo Campazzo*,2020–present,Argentina
4,Argentina,—,Carlos Delfino,2004–2008;2009–2014,Argentina
...,...,...,...,...,...
583,Venezuela,United States,Askia Jones,1994,United States
584,Venezuela,United States,Harold Keeling,1986,United States
585,Venezuela,United States,Donta Smith,2004–2006,United States
586,Venezuela,—,Óscar Torres,2001–2002,Venezuela


In [42]:
# drop the nationality, birthplace and career columns 
tables = tables.drop(columns = ['Nationality[A]', 'Birthplace[B]', 'Career[C]'])
tables

,Player,Country
0,Bruno Fernando*,Angola
1,Julius Hodge,United States
2,Nicolás Brussino,Argentina
3,Facundo Campazzo*,Argentina
4,Carlos Delfino,Argentina
...,...,...
583,Askia Jones,United States
584,Harold Keeling,United States
585,Donta Smith,United States
586,Óscar Torres,Venezuela


In [43]:
# Remove special characters * asterix
# Referenced from Stack Overflow https://stackoverflow.com/questions/38277928/remove-special-characters-in-pandas-dataframe/38277932
tables=tables.replace('\*','',regex=True)
tables

,Player,Country
0,Bruno Fernando,Angola
1,Julius Hodge,United States
2,Nicolás Brussino,Argentina
3,Facundo Campazzo,Argentina
4,Carlos Delfino,Argentina
...,...,...
583,Askia Jones,United States
584,Harold Keeling,United States
585,Donta Smith,United States
586,Óscar Torres,Venezuela


In [44]:
final = pd.merge(all_df, tables, on = 'Player', how = 'left')

In [45]:
final

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season,Country
0,Steven Adams,C,26,OKC,63,63,1680,283,478,0.592,...,376,583,146,51,67,94,122,684,2020,New Zealand
1,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,0.557,...,559,735,368,82,93,204,182,1146,2020,NaN
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,0.493,...,289,392,129,36,87,74,128,1001,2020,NaN
3,Kyle Alexander,C,23,MIA,2,0,13,1,2,0.500,...,1,3,0,0,0,1,1,2,2020,Canada
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,0.368,...,75,84,89,17,8,54,57,267,2020,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,Haywoode Workman,PG,25,WSB,73,56,2034,234,515,0.454,...,191,242,353,87,7,135,162,581,1991,NaN
13560,James Worthy,SF,29,LAL,78,74,3008,716,1455,0.492,...,249,356,275,104,35,127,117,1670,1991,NaN
13561,Howard Wright,PF,23,TOT,15,0,164,19,47,0.404,...,33,45,3,4,5,11,28,54,1991,NaN
13562,A.J. Wynder,PG,26,BOS,6,0,39,3,12,0.250,...,2,3,8,1,0,4,1,12,1991,NaN


In [49]:
final.loc[final['Player'] =="Steven Adams"]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season,Country
0,Steven Adams,C,26,OKC,63,63,1680,283,478,0.592,...,376,583,146,51,67,94,122,684,2020,New Zealand
532,Steven Adams,C,25,OKC,80,80,2669,481,809,0.595,...,369,760,124,117,76,135,204,1108,2019,New Zealand
1061,Steven Adams,C,24,OKC,76,76,2487,448,712,0.629,...,301,685,88,92,78,128,215,1056,2018,New Zealand
1601,Steven Adams,C,23,OKC,80,80,2389,374,655,0.571,...,332,613,86,89,78,146,195,905,2017,New Zealand
2087,Steven Adams,C,22,OKC,80,80,2014,261,426,0.613,...,314,533,62,42,89,84,223,636,2016,New Zealand
2563,Steven Adams,C,21,OKC,70,67,1771,217,399,0.544,...,324,523,66,38,86,99,222,537,2015,New Zealand
3054,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2014,New Zealand


In [50]:
final = final.fillna('United States')

In [51]:
final

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season,Country
0,Steven Adams,C,26,OKC,63,63,1680,283,478,0.592,...,376,583,146,51,67,94,122,684,2020,New Zealand
1,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,0.557,...,559,735,368,82,93,204,182,1146,2020,United States
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,0.493,...,289,392,129,36,87,74,128,1001,2020,United States
3,Kyle Alexander,C,23,MIA,2,0,13,1,2,0.500,...,1,3,0,0,0,1,1,2,2020,Canada
4,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,0.368,...,75,84,89,17,8,54,57,267,2020,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,Haywoode Workman,PG,25,WSB,73,56,2034,234,515,0.454,...,191,242,353,87,7,135,162,581,1991,United States
13560,James Worthy,SF,29,LAL,78,74,3008,716,1455,0.492,...,249,356,275,104,35,127,117,1670,1991,United States
13561,Howard Wright,PF,23,TOT,15,0,164,19,47,0.404,...,33,45,3,4,5,11,28,54,1991,United States
13562,A.J. Wynder,PG,26,BOS,6,0,39,3,12,0.250,...,2,3,8,1,0,4,1,12,1991,United States


## Export 'final' DataFrame to SQLite

In [53]:
engine = create_engine('sqlite:///PlayerStats.db', echo=True)
sqlite_connection = engine.connect()

2021-02-06 13:00:27,491 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-06 13:00:27,506 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:00:27,512 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-06 13:00:27,519 INFO sqlalchemy.engine.base.Engine ()


In [54]:
# Convert DataFrame to SQL
final.to_sql('data',con=engine)

2021-02-06 13:03:08,853 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("data")
2021-02-06 13:03:08,858 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:03:08,882 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("data")
2021-02-06 13:03:08,886 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:03:08,907 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE data (
	"index" BIGINT, 
	"Player" TEXT, 
	"Pos" TEXT, 
	"Age" BIGINT, 
	"Tm" TEXT, 
	"G" BIGINT, 
	"GS" BIGINT, 
	"MP" BIGINT, 
	"FG" BIGINT, 
	"FGA" BIGINT, 
	"FG%" FLOAT, 
	"3P" BIGINT, 
	"3PA" BIGINT, 
	"3P%" FLOAT, 
	"2P" BIGINT, 
	"2PA" BIGINT, 
	"2P%" FLOAT, 
	"eFG%" FLOAT, 
	"FT" BIGINT, 
	"FTA" BIGINT, 
	"FT%" FLOAT, 
	"ORB" BIGINT, 
	"DRB" BIGINT, 
	"TRB" BIGINT, 
	"AST" BIGINT, 
	"STL" BIGINT, 
	"BLK" BIGINT, 
	"TOV" BIGINT, 
	"PF" BIGINT, 
	"PTS" BIGINT, 
	"Season" BIGINT, 
	"Country" TEXT
)


2021-02-06 13:03:08,918 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 13:03:09,170 INFO sqlalchemy.engine.bas